In [1]:
# importing libraries

import pandas as pd
import numpy as np

In [2]:
# loading csv file

df = pd.read_csv(r'dataset_5secondWindow\dataset_5secondWindow.csv')
df.head()

,Unnamed: 0,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,...,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,...,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,...,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,U12
4,4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,...,89.770732,89.770732,89.770732,0.006389,9.000000,9.000000,9.000000,NaN,Car,U1


In [3]:
#first class class of classification
df = df[['Unnamed: 0',
 'id',
 'time',
 'activityrecognition#1',
 'android.sensor.accelerometer#mean',
 'android.sensor.accelerometer#min',
 'android.sensor.accelerometer#max',
 'android.sensor.accelerometer#std',
 'android.sensor.gyroscope#mean',
 'android.sensor.gyroscope#min',
 'android.sensor.gyroscope#max',
 'android.sensor.gyroscope#std',
 'sound#mean',
 'sound#min',
 'sound#max',
 'sound#std',
 'target',
 'user']]

In [4]:
# ordinal encoding

from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()

ct = asarray(df['target'])
df['target'] = oe.fit_transform(ct.reshape(-1,1))
dt = asarray(df['user'])
df['user'] = oe.fit_transform(dt.reshape(-1,1))

In [5]:
# sorting by users
df = df.sort_values(by=['user'])

In [6]:
# dropping the columns
df = df.drop(columns=[ 'user','Unnamed: 0','id' ])

In [7]:
#filling null values
df = df.fillna(0)

In [8]:
# manually splitting train n test without no data leak
dtr = df.iloc[2470:,:]
dts = df.iloc[:2470,:]

In [9]:
# manually splitting train n test with some data leak
dtr_lk = df.iloc[2000:,:]
dts_lk = df.iloc[:2000,:]

In [10]:
# train and test split with no data leak

x_train = dtr.drop(['target'], axis=1)
x_test = dts.drop(['target'], axis=1)

y_train = dtr['target']
y_test = dts['target'] 

# train and test split with data leak

x_train_lk = dtr_lk.drop(['target'], axis=1)
x_test_lk = dts_lk.drop(['target'], axis=1)

y_train_lk = dtr_lk['target']
y_test_lk = dts_lk['target'] 

In [11]:
# scaling data

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

x_train_lk = sc.fit_transform(x_train_lk)
x_test_lk = sc.transform(x_test_lk)

In [12]:
# importing classifiers and metrics

from sklearn.linear_model      import LogisticRegression
from sklearn.svm               import SVC
from sklearn.tree              import DecisionTreeClassifier
from sklearn.ensemble          import ExtraTreesClassifier
from sklearn.ensemble          import AdaBoostClassifier
from sklearn.experimental      import enable_hist_gradient_boosting 
from sklearn.ensemble          import HistGradientBoostingClassifier
from xgboost                   import XGBClassifier
from lightgbm                  import LGBMClassifier
from catboost                  import CatBoostClassifier
from sklearn.ensemble          import RandomForestClassifier
from sklearn.ensemble          import GradientBoostingClassifier
from sklearn.pipeline          import make_pipeline
import time
from sklearn                   import impute, compose, metrics, preprocessing

In [13]:
# training models

classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM": GradientBoostingClassifier(),
  "Skl HistGBM":HistGradientBoostingClassifier(),
  "XGBoost": XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost": CatBoostClassifier(),
  "SVM":  SVC(),
  "log_regression": LogisticRegression()
}

In [14]:
results = pd.DataFrame({'Model': [],  'Accuracy': [], 'Bal Acc.': []})

for model_name, model in classifiers.items():
    start_time = time.time()
    model.fit(x_train, y_train)
    pred = model.predict(x_test)
    total_time = time.time() - start_time
    results = results.append({"Model": model_name,
                              "Accuracy": metrics.accuracy_score(y_test, pred) * 100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred) * 100,
                              "Time": total_time},
                              ignore_index=True)

results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

Learning rate set to 0.084252
0:	learn: 1.4973136	total: 178ms	remaining: 2m 57s
1:	learn: 1.4167852	total: 192ms	remaining: 1m 35s
2:	learn: 1.3509354	total: 207ms	remaining: 1m 8s
3:	learn: 1.2921844	total: 228ms	remaining: 56.7s
4:	learn: 1.2446732	total: 244ms	remaining: 48.5s
5:	learn: 1.1982169	total: 259ms	remaining: 42.9s
6:	learn: 1.1500868	total: 274ms	remaining: 38.9s
7:	learn: 1.1112434	total: 292ms	remaining: 36.3s
8:	learn: 1.0732526	total: 307ms	remaining: 33.8s
9:	learn: 1.0412695	total: 322ms	remaining: 31.9s
10:	learn: 1.0121178	total: 339ms	remaining: 30.4s
11:	learn: 0.9857080	total: 354ms	remaining: 29.1s
12:	learn: 0.9601838	total: 370ms	remaining: 28.1s
13:	learn: 0.9389429	total: 387ms	remaining: 27.3s
14:	learn: 0.9202216	total: 405ms	remaining: 26.6s
15:	learn: 0.9000377	total: 422ms	remaining: 26s
16:	learn: 0.8806739	total: 443ms	remaining: 25.6s
17:	learn: 0.8621415	total: 461ms	remaining: 25.1s
18:	learn: 0.8472671	total: 479ms	remaining: 24.7s
19:	learn: 

C:\Anaconda\envs\strive\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,Model,Accuracy,Bal Acc.,Time
1,Random Forest,56.113360,56.534766,1.700119
2,XGBoost,56.113360,56.855918,1.653787
3,log_regression,55.141700,57.198955,0.289021
4,Extra Trees,54.008097,54.923344,1.041326
5,LightGBM,52.874494,53.196443,0.860043
6,SVM,52.388664,52.843440,2.284827
7,Skl HistGBM,51.740891,52.807891,4.023280
8,Skl GBM,51.417004,51.711777,7.766566
9,CatBoost,51.417004,50.739402,17.592093
10,AdaBoost,48.380567,53.320700,0.882061


In [15]:
results_lk = pd.DataFrame({'Model': [],  'Accuracy': [], 'Bal Acc.': []})

for model_name, model in classifiers.items():
    start_time = time.time()
    model.fit(x_train_lk, y_train_lk)
    pred = model.predict(x_test_lk)
    total_time = time.time() - start_time
    results_lk = results_lk.append({"Model": model_name,
                              "Accuracy": metrics.accuracy_score(y_test_lk, pred) * 100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test_lk, pred) * 100,
                              "Time": total_time},
                              ignore_index=True)

results_ord_lk = results_lk.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord_lk.index += 1
results_ord_lk.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

Learning rate set to 0.084806
0:	learn: 1.5010259	total: 15.1ms	remaining: 15.1s
1:	learn: 1.4195798	total: 29.2ms	remaining: 14.6s
2:	learn: 1.3527972	total: 42.7ms	remaining: 14.2s
3:	learn: 1.2939767	total: 56.4ms	remaining: 14s
4:	learn: 1.2440311	total: 71.5ms	remaining: 14.2s
5:	learn: 1.1974200	total: 86.6ms	remaining: 14.3s
6:	learn: 1.1524741	total: 101ms	remaining: 14.4s
7:	learn: 1.1158967	total: 115ms	remaining: 14.3s
8:	learn: 1.0818286	total: 128ms	remaining: 14.1s
9:	learn: 1.0500860	total: 143ms	remaining: 14.2s
10:	learn: 1.0227903	total: 158ms	remaining: 14.2s
11:	learn: 0.9953144	total: 171ms	remaining: 14.1s
12:	learn: 0.9729624	total: 185ms	remaining: 14.1s
13:	learn: 0.9503826	total: 200ms	remaining: 14.1s
14:	learn: 0.9263322	total: 214ms	remaining: 14.1s
15:	learn: 0.9068496	total: 229ms	remaining: 14.1s
16:	learn: 0.8879500	total: 244ms	remaining: 14.1s
17:	learn: 0.8707888	total: 259ms	remaining: 14.1s
18:	learn: 0.8538414	total: 274ms	remaining: 14.1s
19:	lea

C:\Anaconda\envs\strive\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,Model,Accuracy,Bal Acc.,Time
1,Skl HistGBM,89.950000,90.093971,27.836071
2,Extra Trees,89.150000,89.183966,0.866350
3,LightGBM,88.900000,88.987453,0.617039
4,CatBoost,87.700000,87.753604,15.903059
5,Random Forest,86.850000,86.711003,1.413244
6,Skl GBM,82.450000,83.030399,11.730805
7,XGBoost,80.000000,80.826375,1.802470
8,Decision Tree,75.050000,75.937231,0.073444
9,SVM,66.200000,66.730242,1.870136
10,AdaBoost,64.050000,63.585533,0.538647
